In [12]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output
from flask import Flask


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIXED #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AACcrud import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIXED update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = dash.Dash(__name__)
application = app.server
app.tittle = 'SNHU CS-340 Dashboard - Thomas Brown'

style_data_conditional = [
    {
        "if": {"state": "active"},
        "backgroundColor": "rgba(150, 180, 225, 0.2)",
        "border": "1px solid blue",
    },
    {
        "if": {"state": "selected"},
        "backgroundColor": "rgba(0, 116, 217, .03)",
        "border": "1px solid blue",
    },
]

#Grazioso Salvare’s logo
image_filename = 'GS_logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')



app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.A([html.Img(
        src='data:image/png;base64,{}'.format(encoded_image), style={'width':'10%'})],
        href="https://snhu.edu")),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Thomas Brown'))),
    html.Br(),
    dcc.Dropdown(id='dropdown',
             options = [{'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                        {'label' : 'Mountain or Wilderness Rescue', 
                         'value' : 'Mountain or Wilderness Rescue'},
                        {'label' : 'Disaster Rescue or Individual Tracking', 
                         'value' : 'Disaster Rescue or Individual Tracking'},
                        {'label' : 'No Filter', 'value' : 'No Filter'}], 
             placeholder = 'No Filter',),

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": False} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=50,
        style_table={'height':'400px','overflowY':'auto','overflowX':'auto'},
        tooltip ={i: {
            'value': i,
            'use_with': 'both'
        } for i in df.columns},
        tooltip_delay=0,
        tooltip_duration = None,
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        column_selectable=True,
        row_selectable='single',
        row_deletable=False,
        selected_rows=[],
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it

@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "active_cell")]
)
def update_selected_row_color(active):
    style = style_data_conditional.copy()
    if active:
        style.append(
            {
                "if": {"row_index": active["row"]},
                "backgroundColor": "rgba(150, 180, 225, 0.2)",
                "border": "1px solid blue",
            },
        )
    return style

@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('dropdown', 'value')])
def update_rows(value):

    if (value == 'Water Rescue'):
        df = pd.DataFrame.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'Labrador Retriever Mix'}, {'breed':'Chesapeake Bay Retriever'},
                   {'breed':'Newfoundland'}]}, 
            {'sex_upon_outcome':'Intact Female'}, {'age_upon_outcome_in_weeks':{'$gte':26, '$lte':156}}]}))
        columns = [{'name': i, 'id': i} for i in df.columns]
        data = df.to_dict('records')
        

    elif (value == 'Mountain or Wilderness Rescue'):
        df = pd.DataFrame.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'German Shepherd'}, {'breed':'Alaskan Malamute'},
                   {'breed':'Old English Sheepdog'},{'breed':'Siberian Husky'},{'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$gte':26, '$lte':156}}]}))
        columns = [{'name': i, 'id': i} for i in df.columns]
        data = df.to_dict('records')

    elif (value == 'Disaster Rescue or Individual Tracking'):
        df = pd.DataFrame.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'Doberman Pinscher'}, {'breed':'German Sheperd'},
                   {'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$gte':20, '$lte':300}}]}))
        columns = [{'name': i, 'id': i} for i in df.columns]
        data = df.to_dict('records')

    elif(value == 'No Filter'):
        df = pd.DataFrame.from_records(shelter.read({}))
        columns = [{'name': i, 'id': i} for i in df.columns]
        data = df.to_dict('records')

    return (data, columns)


@app.callback(
    Output("map-id", "children"),
    [
        Input("datatable-id", "derived_virtual_data"),
        Input("datatable-id", "derived_virtual_selected_rows"),
    ],
)
def update_map(derived_virtual_data, selected_row_index):
    dff = df if selected_row_index is None else pd.DataFrame(derived_virtual_data)

    return [
        dl.Map(
            style={"width": "1000px", "height": "500px"},
            center=[
                float(dff.iloc[selected_row_index, 13].values[0]),
                float(dff.iloc[selected_row_index, 14].values[0]),
            ],  
            zoom=10,
            children=[
                dl.TileLayer(id=f"base_layer_id"), 
                # Marker with tool tip and popup
                dl.Marker(
                    position=[
                        float(dff.iloc[selected_row_index, 13].values[0]),
                        float(dff.iloc[selected_row_index, 14].values[0]),
                    ],
                    children=[
                        dl.Tooltip(dff.iloc[selected_row_index, 4]), 
                        dl.Popup(
                            [html.H2("Animal Name"), html.P(dff.iloc[selected_row_index, 9])]
                        ),  
                    ],
                ),
            ],
        )
    ]

# pie chart graph of animal types
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(view_data):
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(view_data)
    breeds = []
    for i in range(0, dff.__len__() - 1):
        breeds.append(str(dff.iloc[i, 4]))
    return [
        dcc.Graph(
            figure={
                'data': [
                    {
                        'labels': breeds,
                        'type': 'pie'
                    }
                ],
                'layout': {
                    'title': 'Breeds',
                }
            }
        )
    ]



if __name__ == '__main__':
    application.run(port=8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Aug/2022 00:07:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:08:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:08:00] "GET /_dash-layout HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [16/Aug/2022 00:08:00] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [16/Aug/2022 00:08:00] "POST /_dash-update-component HTTP/1.1" 200 -


...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [16/Aug/2022 00:22:22] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [16/Aug/2022 00:22:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:22:55] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [16/Aug/2022 00:22:55] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [16/Aug/2022 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:24:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:24:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:24:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:24:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:24:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:24:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:25:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:25:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:25:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2022 00:26:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.